In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch.nn as nn
import torch.optim as optim
import numpy as np


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def train_val_dataset(dataset, val_split=0.1):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    return dict({
        "train" : Subset(dataset, train_idx),
        "val" : Subset(dataset, val_idx)
    })

def createDataLoader():
    mean_values = (0.5, 0.5, 0.5)
    std_values = (0.5, 0.5, 0.5)
    op_size = 32
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.ToTensor(),
        transforms.Normalize(mean_values,std_values ),
        ])

    transform_horizontal = transforms.Compose([
        transforms.RandomResizedCrop(op_size),                                        
        transforms.RandomHorizontalFlip(),                                       
        transforms.ToTensor(),
        ])
    transform_vertical = transforms.Compose([
        transforms.RandomResizedCrop(op_size),                                     
        transforms.RandomVerticalFlip(),                                       
        transforms.ToTensor(),
        ])
    transform_Invert= transforms.Compose([
        transforms.RandomResizedCrop(op_size),                                  
        transforms.RandomInvert(),                                       
        transforms.ToTensor(),
        ])

    dataset_url = "/content/train_local/inaturalist_12K/train"

    img_data = torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_train)
    img_data_hori= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_horizontal)
    img_data_vert= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_vertical)
    img_data_inve= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_Invert)
    img_data = img_data + img_data_inve + img_data_vert + img_data_hori

    img_data = train_val_dataset(img_data)


    X_train=img_data['train']
    X_Valid=img_data['val']


    trainloader = torch.utils.data.DataLoader(X_train, batch_size=128, shuffle=True)
    validationloader = torch.utils.data.DataLoader(X_Valid, batch_size=128, shuffle=False)


    dataiter = iter(trainloader)
    images, labels = dataiter.next()
    return trainloader,validationloader


def accuracy(dataset_itr,model,norm_fact):
    total = 0
    pred_count = 0
    for dataset in dataset_itr:
        X,y=dataset

        X = X.to(device)
        y = y.to(device)

        pred = torch.max(model(X,norm_fact).data,1)[1]

        total+=y.size(0)
        pred_count+=(pred==y).sum().item()
        acc = (100*pred_count)/total
    return acc


In [ ]:
class CnnModel(nn.Module):
    def __init__(self,is_batch_norm,filter_size,file_org,F,d,actication_fun):
        super(CnnModel, self).__init__()
        if is_batch_norm:
            self.cnn_model = nn.Sequential(
                nn.Conv2d(3,filter_size,F),
                nn.BatchNorm2d(int(filter_size)),
                actication_fun,
                nn.MaxPool2d(2,stride=1),  
                
                nn.Conv2d(filter_size,int(filter_size*file_org),F), 
                nn.BatchNorm2d(int(filter_size*(file_org**1))),
                actication_fun,
                nn.MaxPool2d(2,stride=1), 
                
                nn.Conv2d(int(filter_size*file_org),int(filter_size*(file_org**2)),F), 
                nn.BatchNorm2d(int(filter_size*(file_org**2))),
                actication_fun,
                nn.MaxPool2d(2,stride=1), 
                            
                nn.Conv2d(int(filter_size*(file_org**2)),int(filter_size*(file_org**3)),F),    
                nn.BatchNorm2d(int(filter_size*(file_org**3))),
                actication_fun,
                nn.MaxPool2d(2,stride=2), 
                
                nn.Conv2d(int(filter_size*(file_org**3)),int(filter_size*(file_org**4)),int(F)), 
                nn.BatchNorm2d(int(filter_size*(file_org**4))),
                actication_fun,
                nn.MaxPool2d(2,stride=2), 
            
            )
        else:
            self.cnn_model = nn.Sequential(
            nn.Conv2d(3,filter_size,F),
            actication_fun,
            nn.MaxPool2d(2,stride=1),

            nn.Conv2d(filter_size,int(filter_size*file_org),F),           
            actication_fun,
            nn.MaxPool2d(2,stride=1), 
            
            nn.Conv2d(int(filter_size*file_org),int(filter_size*(file_org**2)),F),
            actication_fun,
            nn.MaxPool2d(2,stride=1), 

            nn.Conv2d(int(filter_size*(file_org**2)),int(filter_size*(file_org**3)),F),
            actication_fun,
            nn.MaxPool2d(2,stride=2),

            nn.Conv2d(int(filter_size*(file_org**3)),int(filter_size*(file_org**4)),int(F)),
            actication_fun,
            nn.MaxPool2d(2,stride=2),

            output_dim=x_dim-self.F+1,

            )


        x_dim=32
        y_dim=x_dim-F+1  

        x_dim=y_dim-2   
        y_dim=x_dim-F+1  
        y_dim=y_dim-1 

        x_dim=y_dim
        y_dim=x_dim-F+1 
        y_dim=y_dim-1  

        x_dim=y_dim
        y_dim=x_dim-F+1 
        y_dim=int(y_dim/2) 
        
        x_dim=y_dim
        y_dim=x_dim-F+1  

        y_dim=y_dim/2
        y_dim=int(y_dim)
 
        self.fc_model = nn.Sequential(
           nn.Linear(int(y_dim*y_dim*filter_size*(file_org**4)),120),
           nn.ReLU(),
           nn.Dropout(d),
           nn.Linear(120,10)
        )
    def forward(self, x,batch_norm):
        x = self.cnn_model(x)
        if batch_norm:
            x = self.cnn_model(x)
        return self.fc_model(x.view(x.size(0), -1))

      
def fit(batch_norm,k,file_org,F,max_epochs,batch_size,d,act_fun=nn.ReLU()):
    model = CnnModel(is_batch_norm = batch_norm,
                        filter_size = k,
                        file_org = file_org,
                        F = F,
                        d = d,
                        actication_fun = act_fun).to(device)
    opt = optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    train_loss,val_loss=[],[]
    
    n_iter=np.ceil(8999/batch_size)
    trainloader,validationloader = createDataLoader()
    epoch = 0
    while epoch<max_epochs:
        for key,data in enumerate(trainloader,0):
            X,y=data
            X = X.to(device)
            y = y.to(device)
            opt.zero_grad()

            outputs=model(X,batch_norm) 
            loss=loss_fn(outputs,y)
            loss.backward()

            opt.step()
        
            del X,y,outputs
            torch.cuda.empty_cache()
        
            if(key%100==0):
                print(f"Iter No. : {key}/{n_iter} , loss: {round(loss.item(),2)} ")
        
        
        
        for key,data in enumerate(validationloader,0):
            X,y=data
            X = X.to(device)
            y = y.to(device)

            outputs = model(X,batch_norm)
            loss = loss_fn(outputs,y)

            del X
            del y
            del outputs
            torch.cuda.empty_cache()

        epoch+=1
    val_loss.append(loss.item()) 
    train_loss.append(loss.item()) 
  
    print("Training_accuracy:%.2f" % (accuracy(trainloader,model,batch_norm)))
    print("Validation_accuracy:%.2f" % (accuracy(validationloader,model,batch_norm)))